In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import matplotlib.pyplot as plt
from itertools import combinations
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
%config Completer.use_jedi = False


In [6]:
data_2020 = pd.read_csv('data1.csv')
data_2019 = pd.read_csv('medium_2019.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
data = data_2019.append(data_2020)

In [8]:
data

,Unnamed: 0,article_id,article_url,title,subtitle,claps,responses,reading_time,tag,date
0,0,1,https://betterprogramming.pub/mermaid-create-c...,Mermaid — Create Charts and Diagrams With Mark...,Mermaid is a simple markdown-like…,1200,8,4,Data Visualization,2019-10-30
1,1,2,https://towardsdatascience.com/how-to-graph-a-...,How to graph a Bar Chart Race and realize I do...,My curiosity starts with this type of “Bar…,273,0,4,Data Visualization,2019-10-30
2,2,3,https://medium.com/nightingale/maps-eyes-imagi...,"Maps = Eyes + Imagination, Envisioning the Kno...",Part 4 in a series exploring global information…,126,0,19,Data Visualization,2019-10-30
3,3,4,https://medium.com/deepdreaming/understanding-...,Understanding your data: Principal Component A...,NaN,83,0,11,Data Visualization,2019-10-30
4,4,5,https://medium.com/@mentormelv/add-a-progress-...,Add A Progress Bar to Your Python For Loop,NaN,1,0,1,Data Visualization,2019-10-30
...,...,...,...,...,...,...,...,...,...,...
63067,63067,63068,https://medium.com/@kangaroo_thoughts/how-to-l...,"How to Lose, Keep or Find a Friend",NaN,0,0,4,Family,2020-04-14
63068,63068,63069,https://medium.com/@zartuche/04-14-2020-3b9e26...,04/14/2020,NaN,0,0,0,Family,2020-04-14
63069,63069,63070,https://medium.com/@singh25gaurav/breast-cance...,"<strong class=""markup--strong markup--h3-stron...",NaN,0,0,2,Family,2020-04-14
63070,63070,63071,https://medium.com/@theshanaeast/day-29-an-exc...,DAY 29 (an excerpt from DAY 15 — A BLOG.),NaN,0,0,1,Family,2020-04-14


In [108]:
df = data[(data.tag == 'Religion') | (data.tag == 'Art') | (data.tag == 'Venture-Capital') | (data.tag =='Social Media') | (data.tag =='Technology') | (data.tag =='Business') | (data.tag =='Machine Learning') | (data.tag =='Marketing') | (data.tag =='Javascript') | (data.tag =='Language') | (data.tag == 'Science') | (data.tag == 'Health') | (data.tag == 'Gaming') | (data.tag == 'Fitness') | (data.tag == 'Lifestyle') | (data.tag == 'Culture')]


In [10]:
import nltk
# Download the word corpa with a GUI that opens up
#nltk.download()
##
words = set(nltk.corpus.words.words())
# Download t
def textPreProcess(text):
    # lowercase
    text = text.lower()
    
    #remove tags
    text = re.sub("", "", text)
    
    # Removing non english words
    text = " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() in words or not w.isalpha())
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

textPreProcess('Mermaid is a simple markdown-like hola amigo…')

'mermaid is a simple markdown like '

In [11]:
import unicodedata
import re
def title_processing(text):
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)
    text = unicodedata.normalize("NFKD", text)
    return text

In [109]:
df.loc[:,'pro_title'] =  df.iloc[:,3].apply(textPreProcess)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [110]:
df = df[df.loc[:,'pro_title'].apply(lambda x: len(x)) > 3]

In [111]:
df = df.drop('Unnamed: 0', axis = 1)

In [112]:
df = df[df.subtitle!='0']


In [113]:
df = df[df.subtitle.notna()]

In [114]:
df.loc[:,'title'] = df.loc[:,'title'].apply(title_processing)
df.loc[:, 'title'] = df.loc[:,'title'].apply(lambda text: re.sub("(\\d|\\W)+"," ",text))

In [115]:
df.loc[:,"pro_title"] = df.loc[:,'pro_title'].apply(title_processing)

In [133]:
tf = TfidfVectorizer(analyzer= 'word', stop_words= 'english')
tfidf_matrix = tf.fit_transform((df['tag'])+(df['title']))
tfidf_matrix.shape
tfidf_matrix

<22400x24387 sparse matrix of type '<class 'numpy.float64'>'
	with 110352 stored elements in Compressed Sparse Row format>

In [134]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix)#defining cosine similarity matrix

In [135]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=df['title'], columns=df['title'])#creating a dataframe for cosine similiarity matrix
print('Shape:', cosine_sim_df.shape)
cosine_sim_df.sample(5, axis=1).round(2)

Shape: (22400, 22400)


title,Why We Need Courage to Embrace God s Good News,Cheese Grilled,Things to Do With Used Panties,Dracula Unleashed director Mike Plant was attacked by a wolf,You re Focusing on the Wrong Things
title,,,,,
Calculating String Similarity in Python,0.0,0.0,0.0,0.0,0.0
Decision Tree from Scratch in Python,0.0,0.0,0.0,0.0,0.0
Five Data Science Interview Questions that you must be able to answer,0.0,0.0,0.0,0.0,0.0
Jupyter notebook autocompletion,0.0,0.0,0.0,0.0,0.0
Simple BERT using TensorFlow,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
Staying Productive When Remote Working Notes From a Procrastinator,0.0,0.0,0.0,0.0,0.0
Living with options,0.0,0.0,0.0,0.0,0.0
witch hairs,0.0,0.0,0.0,0.0,0.0


In [136]:
titles = list(df.title)

In [137]:
def recommend(title):
    #titles.index(title)
    index=titles.index(title)
    score = pd.Series(cosine_sim[index]).sort_values(ascending=False)
    s=[]
    s = list(score.iloc[1:6].index)
    z = pd.DataFrame()
    z = pd.concat([pd.DataFrame(df.iloc[[i-1]]) for i in s])
    z[['article_name']] = title
    return z

In [141]:
recommend('Recognizing Handwritten Digits Scikit learn')

,article_id,article_url,title,subtitle,claps,responses,reading_time,tag,date,pro_title,article_name
23273,23274,https://medium.com/luca-chuangs-bapm-notes/usi...,Using Grid Search to tune our machine learning...,Never try hyperparameters one by one by yourself,0,0,2,Machine Learning,2020-03-08,grid search to tune our machine learning,Recognizing Handwritten Digits Scikit learn
27937,27938,https://towardsdatascience.com/takeaways-from-...,Takeaways from OpenAI Five,Last year’s loss to the Champions changed ever...,70,0,12,Machine Learning,2019-04-24,from five,Recognizing Handwritten Digits Scikit learn
30309,30310,https://medium.com/@gomezrickyjay/5-awesome-wa...,Awesome Ways to Skyrocket Your Income This,We were once told that the only way to earn mo...,0,0,8,Lifestyle,2020-10-27,awesome ways to skyrocket your income this,Recognizing Handwritten Digits Scikit learn
23529,23530,https://towardsdatascience.com/beyond-graph-co...,Beyond Graph Convolution Networks,A Dive into Advanced Graph Neural Network Arch...,361,2,10,Machine Learning,2019-07-07,beyond graph convolution,Recognizing Handwritten Digits Scikit learn
21935,21936,https://medium.com/@nikkisharma536/how-to-crea...,How to create custom NER in Spacy,Named Entity Recognition (NER),245,0,3,Machine Learning,2019-11-30,how to create custom in spacy,Recognizing Handwritten Digits Scikit learn


In [144]:
recommended_data = pd.DataFrame()

In [152]:
for i in range(len(titles)):
    individual_data = recommend(titles[i])
    recommended_data = recommended_data.append(individual_data)

In [154]:
def title_processing(text):
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)
    text = unicodedata.normalize("NFKD", text)
    return text


In [155]:
recommended_data.loc[:,'subtitle']= recommended_data.loc[:,'subtitle'].apply(title_processing)


In [157]:
recommended_data.to_csv('recommend.csv', index= False)